# 빅분기 작업형2 Baseline
- 결과를 보려면 print() 함수 필요
- 일반적으로 train, test 두 파일이 제공 $\rightarrow$ train에서 X, y 분리해야함
    - 만약 X_train, X_test, y_train 세 파일이 제공됐다면 위 처럼 X, y를 분리하지 않아도 될 것

## 0. 데이터 로드

In [ ]:
import pandas as pd
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")


print(train.shape)
print(train.head(), "\n")
print(test.shape, "\n")
print(test.head())

# 1. EDA

### 1-1) train EDA

In [ ]:
print(train.info(), "\n")
print(train.isnull.sum(), "\n") # 결측치 확인
print(train.describe(), "\n")

### 1-2) test EDA

In [ ]:
print(test.info(), "\n")
print(test.isnull.sum(), "\n") # 결측치 확인
print(test.describe(), "\n")

In [ ]:
print(train.describe(include="O"), "\n") # train 범주형 변수 unique 확인
print(test.describe(include="O"), "\n") # test 범주형 변수 unique 확인

- train, test에서 범주형 변수의 nuniqie 값 확인하지만.. 아마 시험에서는 값 다르지 않을 것임
- 다르게 되면 처리해야할 게 늘어남 

## 2. 데이터 전처리

### 2-1) 결측치 처리 & ID같은 불필요 칼럼 제거 & train에서 X, y 분리

In [ ]:
# 결측치는 평균보다 이상값에 덜 민감한 중앙값으로 대치
# 데이터에 따라 결측치 방법 고려해보기 (0으로 대치, 평균 대치 ...)
train["칼럼"] = train["칼럼"].fillna(train["칼럼"].median())
test["칼럼"] = test["칼럼"].fillna(test["칼럼"].median())

# 결측치 채워졌나 확인
print(train.isnull().sum(), "\n")
print(test.isnull().sum(), "\n")

# 예측에 불필요한 ID 칼럼 제거
train = train.drop("ID", axis=1)
print(train.head(1), "\n")

# test 데이터 ID 분리
test_ID = test.pop('ID')
print(test.head(1), "\n")

y = train.pop("타켓 칼럼") # X, y  분리
print(y.head(), "\n")

### 2-2) 스케일링
- train에만 fit_transform( ) 해줘야함
- test는 transform( )

In [ ]:
print(train.info(), "\n")

con_df = train.select_dtypes(exclude="object").copy().columns # 수치형 칼럼 선택

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train[con_cols] = scaler.fit_transform(train[con_cols])
train[con_cols] = scaler.transform(train[con_cols])

### 2-3) 인코딩
- 어차피 모델은 랜포, lgbm 비교할 거니까 라벨인코딩, 원핫인코딩 두 방법 다 가능할텐데 범주형 변수 내 nunique 값이 많으면 라벨인코딩 하자

In [ ]:
# 원핫인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# Train 데이터와 Test 데이터의 열 순서를 비교하여 동일한 순서로 정렬
test_data = test[train.columns]

Train 데이터와 Test 데이터의 열 순서가 일치하지 않으면 모델이 올바르게 예측을 수행할 수 없습니다.

모델은 Train 데이터를 학습할 때 피처들의 순서에 의존하여 가중치를 조정하고 학습합니다. 따라서, Train 데이터와 Test 데이터에 동일한 열 순서를 갖도록 정렬해야 모델이 정확한 예측을 수행할 수 있습니다.

범주형 열을 One-Hot 인코딩하여 여러 열로 확장하고, Train 데이터와 Test 데이터를 동일한 열 순서로 정렬하는 것이 중요합니다.

왜 중요한가요?

ChatGPT
Train 데이터와 Test 데이터를 동일한 열 순서로 정렬하는 것이 중요한 이유는 모델이 학습한 피처 순서와 테스트할 때 입력하는 데이터의 피처 순서가 일치해야하기 때문입니다.

One-Hot 인코딩은 범주형 변수를 여러 열로 확장하는 과정에서 새로운 열들이 생성됩니다. 이 때, Train 데이터와 Test 데이터의 열 순서가 일치하지 않으면 모델이 올바르게 예측을 수행할 수 없습니다.

모델은 Train 데이터를 학습할 때 피처들의 순서에 의존하여 가중치를 조정하고 학습합니다. 따라서, Train 데이터와 Test 데이터에 동일한 열 순서를 갖도록 정렬해야 모델이 정확한 예측을 수행할 수 있습니다.

만약 Train 데이터와 Test 데이터의 열 순서가 일치하지 않을 경우, 모델은 학습한 피처의 순서와 다른 피처 순서를 입력으로 받게 되므로 잘못된 결과를 예측할 수 있습니다. 따라서, 정확한 예측을 위해 Train 데이터와 Test 데이터를 동일한 열 순서로 정렬하는 것이 중요합니다.

정리하자면, 모델이 학습한 피처 순서와 테스트할 때 입력하는 데이터의 피처 순서를 일치시키는 것은 모델의 정확도를 보장하기 위한 필수적인 과정입니다.

In [ ]:
# 라벨인코딩
cat_cols = train.select_dtypes(include="object").copy().columns # 범주형 칼럼 선택

from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

print(train.head(), "\n")

## 3) 검증 데이터 분리

## 4) 모델링
- 예측 확률은 이진분류라고 할 때 0(여자), 1(남자)일 확률 이렇게 두 값이 나옴
- 따라서 남자일 확률은 rf.predict_proba(X_val)[:, 1] 이런식으로 지정해줘야함

```python
rf_pred = rf.predict_proba(X_val) # val 셋에서 남자일 예측 '확률'
print(rf_pred)

[[0.64 0.36]
 [0.63 0.37]
 [0.68 0.32]
 ...
 [0.8  0.2 ]
 [0.55 0.45]
 [0.69 0.31]]
 ```

### 4-1) 랜덤포레스트

In [ ]:
# 분류


In [ ]:
# 회귀

### 4-2) lgbm

In [ ]:
# 분류

# 예측 및 예측확률
lgbm_pred = lgbm.predict(X_test)
lgbm_pred_prob = lgbm.predict_proba(X_test)

# ROC-AUC
lgbm_auc = roc_auc_score(y_test, lgbm_pred_prob[:,1])

In [ ]:
# 회귀

## 5) 제출 : df & csv 저장